In [1]:
###Python Packages###

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib.colors import LogNorm
import math as mth
import timeit
import Double_Null_utils as dnu
from scipy import optimize
import tables

#!jupyter-theme -t onedork

In [2]:
###Initial Values###

M0=1.0
Q=0.95
Lambda=0.0000
scalarfield=False
vscalarfield=True


In [3]:
Tol=1.0
N=10
Nul=10
Nup=10**(80)#10**(20)#

scal=1
umax=16
vmax=1000


ru0=5.0
dr0v=.4

bdytype="stan"
scaltostan=False
solveaffine=False

Elist=[1]
#Elist=[1,2,4,8,16]

#uloc=[1/8,1/4,3/8,1/2,5/8]
#uloc=[1/5,2/5,3/5,4/5,9/10]
uloc=[8.2/10]

In [4]:
###Defining Needed Constants###

scalf=float(scal)
u0=0.0
v0=0.0
rv0=ru0
phiu0=0.0
phiv0=0.0

if M0==0.0:
    du00=1/N
    du0=du00
else:
    du00=M0/N
    du0=du00
dv00=du00
dv0=du0


def fr(r,M0,Q,Lambda):
    return 1-2*M0/r+Q**2.0/r**2.0-Lambda*r**2.0/3.0

sol = optimize.root(fr,[0.687,1.3,50.0],args=(M0,Q,Lambda), method='hybr')
rminus=sol.x[0]
rplus=sol.x[1]
rcosm=sol.x[2]

wHD=True 

print("r+: "+str(rplus))
print("r-: "+str(rminus))
print("rc: "+str(rcosm))

###Predicting Run time###

###timer
start = timeit.default_timer()

Nu=Nul*umax*100#int(umax/du0)
Nv=int(vmax/dv0)
#print("Number of points for lowest iteration is "+str(Nu*Nv)+","+str(Nu)+"X"+str(Nv))
print("Number of points for highest iteration is "+str(Nu*Nv*max(Elist)**2)+","+str(Nu*max(Elist))+"X"+str(Nv*max(Elist)))

numpoints=0.0
for i in range(0,len(Elist)):
    numpoints=Elist[i]**2.0*Nu*Nv+numpoints
print("Total number of points is "+str(int(numpoints)))

predtime=numpoints/(78000)

print("Runtime: about "+str(format(predtime/60,'.2f'))+" minutes")


r+: 1.31224989992
r-: 0.687750100079
rc: 1.31224990002
Number of points for highest iteration is 160000000,16000X10000
Total number of points is 160000000
Runtime: about 34.19 minutes


In [5]:
##################################
###Applying Boundary Conditions###

Emax=max([Elist])

rnpf=np.zeros((2,Nv))
signpf=np.zeros((2,Nv))
phinpf=np.zeros((2,Nv))
#dphinpuf=np.zeros((Nu),len(Elist))) 
massnpf=np.zeros((2,Nv))


#tempsignpu=np.zeros((Nu*max(Elist),Nv*max(Elist)))




rnpf[0], signpf[0], phinpf[0] = dnu.boundaryv(scal,bdytype,Nv,ru0,dr0v,dv0,vmax,M0,Q,scalarfield)


print(len(rnpf))
print(rnpf)
#print(rnpf[1])
print()
#print(rnpf)

2
[[   5.      5.04    5.08 ...,  404.88  404.92  404.96]
 [   0.      0.      0.   ...,    0.      0.      0.  ]]



In [6]:
#drunp=np.empty((Nu))*np.nan
#dsigunp=np.empty((Nu))*np.nan


A=.115
u1=9.5
u2=10

###Applying Propagation Algorithm###

urange=[0.0]
drunp=np.array([0.0])
dsigunp=np.array([0.0])
dphinpuf=np.array([0.0])


drunp[0]=-mth.exp(signpf[0][0])/(4.0*dr0v)*(1-2*M0/ru0+Q**2.0/ru0**2.0+Lambda*ru0**2/3)
if bdytype=="edd":
    #dsigunp[0]=2.0*(M0*rnpf[0]-Q**2.0)/(rnpf[0]*(Q**2.0+rnpf[0]*(-2*M0+rnpf[0])))*drunp[0]
    dsigunp[0]=2*(3*Q**2-3*M0*r+r**4*Lambda)/(r*(-3*Q**2+r*(6*M0-3*r+r**3*Lambda)))
else:
    dsigunp[0]=0.0

i=0
dv=dv0
du0=M0/N
if bdytype=="edd":
    bdyvalue=1.0
else:
    bdyvalue=0.0
 
atom = tables.Float64Atom()

rnpfile = tables.open_file('rnp.h5', mode='w',encoding="utf8") 
rnp = rnpfile.create_earray(rnpfile.root, 'data', atom, (0, Nv))

signpfile = tables.open_file('signp.h5', mode='w',encoding="utf8") 
signp = signpfile.create_earray(signpfile.root, 'data', atom, (0, Nv))
phinpfile = tables.open_file('phinp.h5', mode='w',encoding="utf8") 
phinp = phinpfile.create_earray(phinpfile.root, 'data', atom, (0, Nv))
massnpfile = tables.open_file('massnp.h5', mode='w',encoding="utf8") 
massnp = massnpfile.create_earray(massnpfile.root, 'data', atom, (0, Nv))
    
 
#rtemp=rnpf[0]   
#print(rtemp)
rnp.append(rnpf[0].reshape(1,Nv))
signp.append(signpf[0].reshape(1,Nv))
phinp.append(phinpf[0].reshape(1,Nv))
massnp.append(massnpf[0].reshape(1,Nv))

#print(rnp)
    
while max(urange)<umax and i<20000:
    
    #rnpf=np.insert(rnpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #phinpf=np.insert(phinpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #signpf=np.insert(signpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #massnpf=np.insert(massnpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #drunp=np.insert(drunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dsigunp=np.insert(dsigunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dphinpuf=np.insert(dphinpuf,(1),np.empty((1))*np.nan).reshape(i+2)
    
    drunp=np.append(drunp,np.nan)
    dsigunp=np.append(dsigunp,np.nan)
    dphinpuf=np.append(dphinpuf,np.nan)
    
    
    du=abs(du0*Tol/mth.exp(np.nanmax(signpf[0,:])))
    
    
    if du<1/Nup*1.1:
        du=1/Nup
     
    
    if du>=1/Nul:
        #print('r='+str(rnpf[i][0]))
        if rnpf[1][0]<rplus:
            du=1/(10*Nul)
            #print(rplus)
        else:
            du=1/Nul
    
    
    print("%.64f" % urange[i])
    print("%.80f" % du)
    
    if urange[i]>u1 and urange[i]<u2 and vscalarfield==True:
        u=urange[i]+du
        phinpf[1][0]=A*64*(u-u1)**3.0*(u2-u)**3.0/(u2-u1)**6.0
        dphinpuf[i]=192*A*(u-u1)**2.0*(u-u2)**2.0*(-2*u+u1+u2)/(u1-u2)**6.0 
    else:
        phinpf[1][0]=0.0
        dphinpuf[i+1]=0.0
    
    #print(drunp,dsigunp,dphinpuf)
    
    
    rnpf[1][0]=rnpf[0][0]+du*drunp[i]
    drunp[i+1]=drunp[i]+du*(drunp[i]*dsigunp[i]-rnpf[0][0]*dphinpuf[i]**2.0)
    
    
    signpf[1][0]=signpf[0][0]+du*dsigunp[i]
    dsigunp[i+1]=2.0*(M0*rnpf[1][0]-Q**2.0)/(rnpf[1][0]*(Q**2.0+rnpf[1][0]*(-2*M0+rnpf[1][0])))*drunp[i+1]*bdyvalue
    
    
    
    #print(rnpf[1][0])
    for j in range(0,Nv-1):
        answer=dnu.x4giveralt(0,j,du,dv,rnpf,phinpf,signpf,Q,Lambda)
        rnpf[1][j+1]=answer[0]
        phinpf[1][j+1]=answer[1]
        signpf[1][j+1]=answer[2]
        massnpf[1][j+1]=answer[3]
        
        #drunp[1][j+1]=answer[4]
        #dsigunp[1][j+1]=answer[5]
            
    #print(len(rnpf[1]))
    
    rnp.append(rnpf[1].reshape(1,Nv))
    signp.append(signpf[1].reshape(1,Nv))
    phinp.append(phinpf[1].reshape(1,Nv))
    massnp.append(massnpf[1].reshape(1,Nv))
    
    
    rnpf[0,:]=rnpf[1,:]
    phinpf[0,:]=phinpf[1,:]
    signpf[0,:]=signpf[1,:]
    massnpf[0,:]=massnpf[1,:]
    #drunp=np.insert(drunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dsigunp=np.insert(dsigunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dphinpuf=np.insert(dphinpuf,(1),np.empty((1))*np.nan).reshape(i+2)

    urange.append(urange[i]+du) 
    
    if rnpf[1][0]<0.0 or np.isnan(du):
        break
   
    dumaxlist=[]
    i+=1
    
    #if i>1000:
        #break


0.0000000000000000000000000000000000000000000000000000000000000000
0.01000000000000000020816681711721685132943093776702880859375000000000000000000000
0.0100000000000000002081668171172168513294309377670288085937500000
0.09995900370560850678458564289030618965625762939453125000000000000000000000000000
0.1099590037056085017885820320771017577499151229858398437500000000
0.09954599803295044457840390350611414760351181030273437500000000000000000000000000
0.2095050017385589602447737433976726606488227844238281250000000000
0.09912880637039263309606695884212967939674854278564453125000000000000000000000000
0.3086338081089515794630528944253455847501754760742187500000000000
0.09870739568391070917208196533465525135397911071777343750000000000000000000000000
0.4073412037928623163907104753889143466949462890625000000000000000
0.09828173348883334492498420331685338169336318969726562500000000000000000000000000
0.5056229372816956058045434474479407072067260742187500000000000000
0.0978517878977165078691058397453

4.7058149476650443787661970418412238359451293945312500000000000000
0.07122172449309083219759486382827162742614746093750000000000000000000000000000000
4.7770366721581352109637919056694954633712768554687500000000000000
0.07057008332590476717971483822111622430384159088134765625000000000000000000000000
4.8476067554840396311988115485291928052902221679687500000000000000
0.06991500726129902532068882692328770644962787628173828125000000000000000000000000
4.9175217627453386981528637988958507776260375976562500000000000000
0.06925660008158404656875006821792339906096458435058593750000000000000000000000000
4.9867783628269224394102820951957255601882934570312500000000000000
0.06859497014010178250753568818254279904067516326904296875000000000000000000000000
5.0553733329670240692621518974192440509796142578125000000000000000
0.06793023037005579967129165197547990828752517700195312500000000000000000000000000
5.1233035633370800354668972431682050228118896484375000000000000000
0.0672624982844675228799147248537

7.6146355382336858852454497537110000848770141601562500000000000000
0.03428835691242908961795521349813498090952634811401367187500000000000000000000000
7.6489238951461153703803574899211525917053222656250000000000000000
0.03368847807605972882738498697108298074454069137573242187500000000000000000000000
7.6826123732221747175685777619946748018264770507812500000000000000
0.03309441748673607602837520857974595855921506881713867187500000000000000000000000
7.7157067907089107450246956432238221168518066406250000000000000000
0.03250629605055645882094594867339765187352895736694335937500000000000000000000000
7.7482130867594669609843549551442265510559082031250000000000000000
0.03192422868300746896519370920941582880914211273193359375000000000000000000000000
7.7801373154424746658719413971994072198867797851562500000000000000
0.03134832421131124313706806105983559973537921905517578125000000000000000000000000
7.8114856396537861726869778067339211702346801757812500000000000000
0.0307786852927878357144564347436

8.7827697670939919305510557023808360099792480468750000000000000000
0.01112180722995936149599671693977143149822950363159179687500000000000000000000000
8.7938915743239505218298290856182575225830078125000000000000000000
0.01087617575755843099538644480617222143337130546569824218750000000000000000000000
8.8047677500815098028397187590599060058593750000000000000000000000
0.01063561985089318123742785360263951588422060012817382812500000000000000000000000
8.8154033699324028106047990149818360805511474609375000000000000000
0.01040005625722965031110245348600074066780507564544677734375000000000000000000000
8.8258034261896316508000381872989237308502197265625000000000000000
0.01016940181854961849916563210172171238809823989868164062500000000000000000000000
8.8359728280081810680712806060910224914550781250000000000000000000
0.00994357355421427759056829387418474652804434299468994140625000000000000000000000
8.8459164015623947818767192075029015541076660156250000000000000000
0.0097224887389305986173937412786

9.1363932221085129015136772068217396736145019531250000000000000000
0.00317406804604616551990226014368090545758605003356933593750000000000000000000000
9.1395672901545594868366606533527374267578125000000000000000000000
0.00310191238576340913568607504657848039641976356506347656250000000000000000000000
9.1426692025403237806813194765709340572357177734375000000000000000
0.00303139327634599151095606828221207251772284507751464843750000000000000000000000
9.1457005958166703862843860406428575515747070312500000000000000000
0.00296247407746676270218211257656548696104437112808227539062500000000000000000000
9.1486630698941375783306284574791789054870605468750000000000000000
0.00289511892869596075550386871100272401235997676849365234375000000000000000000000
9.1515581888228343387936547514982521533966064453125000000000000000
0.00282929273586587232136579928010178264230489730834960937500000000000000000000000
9.1543874815587003013206413015723228454589843750000000000000000000
0.0027649611574616997367037640742

...

9.2760637292480279114670338458381593227386474609375000000000000000
0.00000000000002810568506631839429465763178995832890778687473964936316406237892807
9.2760637292480563331764642498455941677093505859375000000000000000
0.00000000000003373984039826787635276552881982775698705258674015006192803411977366
9.2760637292480900839564128546044230461120605468750000000000000000
0.00000000000004050414757382868326736952312437377484306529409896491245035576866940
9.2760637292481309401637190603651106357574462890625000000000000000
0.00000000000004862563203069639623428982763981540413605149028586360770987084833905
9.2760637292481789017983828671276569366455078125000000000000000000
0.00000000000005837180858723975601940641199667197038945982978663806761687737889588
9.2760637292482375215740830753929913043975830078125000000000000000
0.00000000000007007047445820701820502698727509144287326110778213461571795050986111
9.2760637292483067994908196851611137390136718750000000000000000000
0.0000000000000841148466609055514

9.2760637324366452816093442379496991634368896484375000000000000000
0.00000000063562767730311410883911597309451138593239249985344940796494483947753906
9.2760637330722737203814176609739661216735839843750000000000000000
0.00000000076216681639556199572768174915217941067790974329909658990800380706787109
9.2760637338344409386081679258495569229125976562500000000000000000
0.00000000091385835340799588750159579981081729715963746230045217089354991912841797
9.2760637347483001491355025791563093662261962890625000000000000000
0.00000000109569239199478623210412933967287246739275019535853061825037002563476562
9.2760637358439925748143650707788765430450439453125000000000000000
0.00000000131364051271298017154071807889971837401965615299559431150555610656738281
9.2760637371576333265466018929146230220794677734375000000000000000
0.00000000157486407097527552745736983714087281471805113142181653529405593872070312
9.2760637387324980096536819473840296268463134765625000000000000000
0.0000000018879454697004085856716

9.2761274474383252908182839746586978435516357421875000000000000000
0.00001234189786971244380217073433048113884069607593119144439697265625000000000000
9.2761397893361952071700216038152575492858886718750000000000000000
0.00001471628152693598425525527834123451498271606396883726119995117187500000000000
9.2761545056177219237270037410780787467956542968750000000000000000
0.00001754462187175249469829781123753065230630454607307910919189453125000000000000
9.2761720502395927923089402611367404460906982421875000000000000000
0.00002090992681236327778698108614818806927360128611326217651367187500000000000000
9.2761929601664050437648256774991750717163085937500000000000000000
0.00002491551142449432469584864879852403873883304186165332794189453125000000000000
9.2762178756778297383789322338998317718505859375000000000000000000
0.00002968214870356228600890152735658489291381556540727615356445312500000000000000
9.2762475578265330966587498551234602928161621093750000000000000000
0.0000353527321062105712558623626

9.8341935332031020777776575414463877677917480468750000000000000000
0.07766808346498409176827948385835043154656887054443359375000000000000000000000000
9.9118616166680855172899100580252707004547119140625000000000000000
0.08536164660061605624630232114213868044316768646240234375000000000000000000000000
9.9972232632687010323024878744035959243774414062500000000000000000
0.09252647752563800120917392177943838760256767272949218750000000000000000000000000
10.0897497407943390612672374118119478225708007812500000000000000000
0.09807853179725577186243157257194980047643184661865234375000000000000000000000000
10.1878282725915951800743641797453165054321289062500000000000000000
0.01000000000000000020816681711721685132943093776702880859375000000000000000000000
10.1978282725915949669115434517152607440948486328125000000000000000
nan


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:69: RuntimeWarning: All-NaN slice encountered


In [7]:
rnpfile.close()
signpfile.close()
phinpfile.close()
massnpfile.close()

In [8]:
Nu=i+2

print(Nu)
#rnpf[i+1][j+1]=answer[0]
#phinpf[i+1][j+1]=answer[1]
#signpf[i+1][j+1]=answer[2]
#massnpf[i+1][j+1]=answer[3]
#drunp[i+1][j+1]=answer[4]
#dsigunp[i+1][j+1]=answer[5]

7267


In [9]:
#print(rnpf[1,0])

In [10]:
#print(dnu.x4giveralt(0,0,du,dv,rnpf,phinpf,signpf,Q,Lambda))

In [11]:
#print(rnpf)

In [12]:
vrange=np.arange(0,vmax,dv0)
#print(urange)

In [13]:
np.save('rnputil',np.array([Nu*Nv,Nu,Nv,ru0,dr0v,M0,Q,Lambda,wHD]))
np.save('urange',urange)
np.save('vrange',vrange)

In [14]:
stop = timeit.default_timer()

In [15]:
acttime=stop - start

numpoints=Nu*Nv



print(str(acttime)+' seconds')
print(str((acttime)/60)+' minutes')
print(str((acttime)/3600)+' hours')

print(str(numpoints)+' points')

print(str((numpoints)/(stop - start))+' points per second') 

exttimef=(acttime-predtime)/60

exttime=format(abs((acttime-predtime)/60),'.2f')
if exttimef>0.0:
    print("Took "+str(exttime)+" more minutes")
if exttimef<0.0:
    print("Took "+str(exttime)+" less minutes")
if exttimef==0.0:
    print("Took exactly the right time!!!!")

912.323910915977 seconds
15.205398515266284 minutes
0.2534233085877714 hours
72670000 points
79653.72728972873 points per second
Took 18.98 less minutes
